# Introduction

The experimentation pipeline will be the following: We'll take a baseline configuration detailed below, and we're going to perform experiments in diferent aspects of the whole pipeline. As we make those experiments, we'll update the baseline keeping the balance with the best results from the previous experiments and hyperparameters that don't compromise the experimentation speed. We're going to use the following baseline for experimentation:
- Descriptor: SIFT, as the common algorithm for feature extraction in classic literature
- Number of features: 1000
- Size of codebook: 512 (For initial fast clustering experimentation)
- Normalization: L2 in histograms
- Spatial Pyramids: No pyramid (no level)
- Dimensionality reuction: None (first we try with the whole data)
- Classifier: Logistic Regression (fast and easy hyperparameters for base experimentation)


With this pipeline, we're going to try:
- Number of local features in descriptors
- SIFT / AKAZE / ORB / Dense SIFT
- Spatial pyramids vs no spatial pyramids
- Try normalization and scale
- Size of codebook
- Try dimensionality reduction
- Classifiers: Logistic Regression vs SVM vs KNN
- Fisher Vectors





# Experiments

In [ ]:
from main import *
from bovw import *

In [ ]:
import os
import random
import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

In [ ]:
data_train = Dataset(ImageFolder="../data/places_reduced/train")
data_test = Dataset(ImageFolder="../data/places_reduced/val")

len(data_train), len(data_test)

In [ ]:
random.shuffle(data_train)
random.shuffle(data_test)

In [ ]:
sample_idx = 76
print(data_train[sample_idx][1])
plt.imshow(np.array(data_train[sample_idx][0]))
plt.show()

In [ ]:
train_class_counter = Counter(entry[1] for entry in data_train)
test_class_counter = Counter(entry[1] for entry in data_test)

classes = sorted(train_class_counter.keys())  # or union with test if needed
train_counts = [train_class_counter[c] for c in classes]
test_counts = [test_class_counter[c] for c in classes]

width = 0.4
x = range(len(classes))

plt.bar([xi - width/2 for xi in x], train_counts, width=width, label="Train")
plt.bar([xi + width/2 for xi in x], test_counts, width=width, label="Test")

plt.xticks(x, classes)
plt.ylabel("Frequency")
plt.title("Class Distribution: Train vs Test")
plt.legend()
plt.tight_layout()
plt.show()

All classes have 800 training images except for class 4 ("industrial and construction"), which has 700.

All classes have 200 test samples.

## A) Descriptors (SIFT vs AKAZE vs ORB) and number of features

Comparision of descriptors (SIFT vs AKAZE vs ORB) while trying different hyperparameters on the descriptors (including the number of features)

In [ ]:
bovw_params = {
    "detector_type": "DSIFT",
    "codebook_size": 100, 
    "detector_kwargs": {"nfeatures": 100},
    "dense_kwargs": {"step": 16, "size": 16}
}

classifier_cls = SVC
classifier_params = {
    "kernel": 'rbf', 
}

scores = cross_validate_bovw(
    data_train,
    bovw_kwargs=bovw_params,
    classifier_cls=classifier_cls,
    classifier_kwargs=classifier_params
)

scores.test.accuracy.mean

## B) Density vs Non Density (SIFT vs Dense SIFT)

ATENCIÓN: AQUÍ ASUMO QUE SIFT SERÁ EL QUE FUNCIONARÁ MEJOR, CANVIAR SI NO ES EL CASO

Since SIFT was the best descriptor, we compare it with Dense SIFT

## C) Size of codebook

Try for different codebook sizes (k) 

## D) Spatial Pyramids

Comaprision between using and not using spatial pyramids

## E) Classifiers and Normalization

Since the way the data is normalized and preprocessed can depend on the classifier, we're going to try this along the different classifiers and it's hyperparameter search for Logistic Regression, SVM, KNN

## F) Dimensionality Reduction

We're going to try PCA, SVD, LDA, t-SNE

## G) Fisher Vectors

Finally, we're going to do a fisher vector approach